In [1]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk
from nltk.corpus import stopwords as stop_words

# nltk.download('stopwords')

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark import SparkContext, SparkConf
import pyLDAvis as vis
import numpy as np
from random import sample
import pandas as pd


from contextualized_tm.subset import get_data_sample

/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [3]:
DATA_PATH = '../../../notebooks/MADE/data.parquet'  # Input path to dataset in parquet format

In [4]:
spark = SparkSession.builder.appName("CombinedTM").getOrCreate()

22/10/23 02:19:27 WARN Utils: Your hostname, PK resolves to a loopback address: 127.0.1.1; using 172.28.78.27 instead (on interface eth0)
22/10/23 02:19:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/23 02:19:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
ids, abstracts = get_data_sample(spark=spark, data_path=DATA_PATH, sample_frac=0.01)

In [6]:
documents = [line.strip() for line in abstracts]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [7]:
len(preprocessed_documents)

29205

In [8]:
tp = TopicModelDataPreparation("allenai/scibert_scivocab_uncased")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

No sentence-transformers model found with name /home/ippk93/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ippk93/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly 

Batches:   0%|          | 0/147 [00:00<?, ?it/s]

In [9]:
tp.vocab[:10]

['ability',
 'able',
 'abstract',
 'abstraction',
 'academic',
 'access',
 'according',
 'account',
 'accuracy',
 'accurate']

In [10]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=40, num_epochs=10)
ctm.fit(training_dataset) # run the model

Epoch: [10/10]	 Seen Samples: [292050/292050]	Train Loss: 459.34648631441536	Time: 0:00:03.440863: : 10it [00:34,  3.42s/it]
Sampling: [20/20]: : 20it [00:37,  1.85s/it]


In [12]:
# Uncomment if you want to save the model
# ctm.save(models_dir="./")

In [13]:
lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

Sampling: [10/10]: : 10it [00:18,  1.81s/it]


In [14]:
ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and 

In [16]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

Sampling: [5/5]: : 5it [00:09,  1.83s/it]


In [17]:
topics_list = []
for i in range(len(topics_predictions)):
    topic_number = np.argmax(topics_predictions[i])
    topics = sample(ctm.get_topic_lists()[topic_number], 5)
    topics_list.append(topics)
# topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [19]:
id_tags = [{"old_id": ids[i], "tags": topics_list[i]} for i in range(len(topics_list))]

In [20]:
df = pd.DataFrame(id_tags)
df.head()

,old_id,tags
0,556fa9182401b4b38c237689,"[search, problem, set, problems, time]"
1,556fa3812401b4b38c237516,"[model, execution, time, models, design]"
2,556fa4312401b4b38c237545,"[scheme, network, sensor, ad, networks]"
3,556facc82401b4b38c23778d,"[images, method, video, frame, coding]"
4,556fb32a2401b4b38c237954,"[privacy, web, information, services, service]"


In [21]:
ctm.get_topic_lists()[0]

['language',
 'software',
 'semantics',
 'knowledge',
 'formal',
 'specification',
 'development',
 'process',
 'logic',
 'languages']

In [22]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location/places/related things

['model', 'system', 'control', 'based', 'time']

In [25]:
print(unpreprocessed_corpus[topic_number])

Dynamic Voltage and Frequency Scaling (DVFS) is commonly used to save energy in computing systems. However, when it comes to parallel programs, existing DVFS controllers only reduce frequency while or before waiting in blocking communications. As a consequence, energy savings are only possible for the program tasks out of the critical path and when the workload is imbalanced. We propose a new runtime DVFS controller, FoREST-mn. It allows to take advantage of both the low CPU usage of some program phases as well as communication slack to save more energy with parallel programs. The DVFS control then becomes more complex, but energy savings are even obtained when the workload is balanced. The resulting slowdown on programs is carefully controlled and constrained by a user-defined threshold. We implemented the presented strategies and evaluated it on 4 compute nodes totaling 64 cores. FoREST-mn is able to perform significant CPU energy savings on the NAS programs, up to 34 % on MG, while 

### Save

In [164]:
# Uncomment if you want to save the data

# import fastparquet
# fastparquet.write('id_tags.parquet', df, file_scheme='hive', write_index=False)

In [165]:
# pf = fastparquet.ParquetFile('id_tags.parquet', )
# pf.to_pandas(index=False)

,old_id,tags
0,556f9f9e2401b4b38c2373f0,"[video, bit, compression, coding, proposed]"
1,556fa3b02401b4b38c237522,"[mutual, confidence, redundant, phases, window]"
2,556fa54a2401b4b38c23758c,"[robots, robot, human, force, trajectory]"
3,556fa13b2401b4b38c23746d,"[camera, robots, motion, position, vehicle]"
4,556fa5c72401b4b38c2375b7,"[facial, deep, features, trained, convolutional]"
...,...,...
437994,6051d8e69e795eb49a3cb838,"[real, model, series, time, algorithm]"
437995,6054328e9e795e40330e1fa2,"[texture, object, image, images, scene]"
437996,60545df19e795e4033115a30,"[smart, systems, sensor, iot, monitoring]"
437997,6058404b9e795e4ac8d1766d,"[graphs, connected, every, edge, cycles]"


### Evaluation

In [39]:
from contextualized_topic_models.evaluation.measures import CoherenceUMASS, CoherenceNPMI

texts = [doc.split() for doc in unpreprocessed_corpus]

umass = CoherenceUMASS(texts=texts, topics=ctm.get_topic_lists(10))
npmi = CoherenceNPMI(texts=texts, topics=ctm.get_topic_lists(10))
umass.score(), npmi.score()

(-3.196950714625764, 0.019660956053620143)